In [1]:
import sys
sys.path.append("..")

from libs.InteractivePitchUI import *
from libs.data_manipulation import *
from libs.dim_reduction import *
from libs.feature_generation import *
from libs.clustering import *
from libs.convex_hull import *
from libs.alpha_shape import *
from libs.weight_generator import *
from libs.similar_movement import *
from libs.Video_Player import *

/home/martin/.local/lib/python3.10/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (2.2.3) or chardet (None)/charset_normalizer (2.1.0) doesn't match a supported version!
  warnings.warn(


In [2]:
matches = compile_team_tracking_data_with_labels("../data", "Denmark", "../data/Labelled_ground_truths.csv")




Index(['Time [s]_team', 'half_team', 'minute_team', 'Period_team', 'home_6_x',
       'home_6_y', 'home_18_x', 'home_18_y', 'home_1_x', 'home_1_y',
       'home_10_x', 'home_10_y', 'home_3_x', 'home_3_y', 'home_2_x',
       'home_2_y', 'home_19_x', 'home_19_y', 'home_21_x', 'home_21_y',
       'home_9_x', 'home_9_y', 'home_17_x', 'home_17_y', 'home_23_x',
       'home_23_y', 'home_20_x', 'home_20_y', 'home_15_x', 'home_15_y',
       'home_8_x', 'home_8_y', 'home_5_x', 'home_5_y', 'home_12_x',
       'home_12_y', 'ball_z_team', 'Time [s]', 'half', 'minute', 'Period',
       'away_10_x', 'away_10_y', 'away_2_x', 'away_2_y', 'away_17_x',
       'away_17_y', 'away_22_x', 'away_22_y', 'away_9_x', 'away_9_y',
       'away_1_x', 'away_1_y', 'away_20_x', 'away_20_y', 'away_21_x',
       'away_21_y', 'away_11_x', 'away_11_y', 'away_13_x', 'away_13_y',
       'away_6_x', 'away_6_y', 'away_7_x', 'away_7_y', 'away_5_x', 'away_5_y',
       'away_14_x', 'away_14_y', 'away_23_x', 'away_23_y', 'away_1

In [3]:
df = extract_one_match(matches,4)

In [4]:
df.head()
df.columns

Index(['match_id', 'Time [s]', 'Time [s]_team', 'half_team', 'minute_team',
       'Period_team', 'home_6_x', 'home_6_y', 'home_18_x', 'home_18_y',
       ...
       'away_25_y', 'home_26_x', 'home_26_y', 'away_3_x', 'away_3_y',
       'away_15_x', 'away_15_y', 'Label', 'Time[s]', 'Competition'],
      dtype='object', length=101)

In [5]:
df = df[df["match_name"].isin(["Denmark_England","Slovenia_Denmark"])]




In [14]:
df.match_name.unique()

array(['Slovenia_Denmark', 'Denmark_England'], dtype=object)

In [6]:
func_control = lambda x, _ = 0 :1
func_inverse = inverse_weighting
func_linear = linear_weighting
func_exp = inverse_exponential_weighting

In [7]:
df.Label.unique()

array(['Missing', 'run-into-space', 'breakthrough-wb',
       'breakthrough-space', 'breakthrough-even'], dtype=object)

In [8]:
import datetime


def process_situations(df):
    # Dictionary to store recommendations
    recommendations_dict = {}

    # Step 2: Extract relevant rows (all rows with a label not equal to "Missing")
    labeled_rows = df[df['Label'] != "Missing"]
    relevant_indices = labeled_rows.index.tolist()
    print(relevant_indices)
    # Step 2: Process each relevant index
    # Step 2: Process each relevant index
    for relevant_index in relevant_indices:
        # Step 3: Apply Wasserstein distance for various functions upfront
        finished_indices_control = most_similar_with_wasserstein_closed_interval(relevant_index, df, func_control)
        finished_indices_linear = most_similar_with_wasserstein_closed_interval(relevant_index, df, func_linear)
        finished_indices_inverse = most_similar_with_wasserstein_closed_interval(relevant_index, df, func_inverse)
        finished_indices_exp = most_similar_with_wasserstein_closed_interval(relevant_index, df, func_exp)

        # Convert to numpy arrays
        finished_indices_control = np.array(finished_indices_control)
        finished_indices_exp = np.array(finished_indices_exp)
        finished_indices_linear = np.array(finished_indices_linear)
        finished_indices_inverse = np.array(finished_indices_inverse)

        # Step 4: Filter indices based on the time window (±480 seconds)
        time_window_filter = lambda indices: indices[
            (indices > relevant_index + 480) | 
            (indices < relevant_index - 480)
        ]

        finished_indices_control = time_window_filter(finished_indices_control)[:100]
        finished_indices_exp = time_window_filter(finished_indices_exp)[:100]
        finished_indices_linear = time_window_filter(finished_indices_linear)[:100]
        finished_indices_inverse = time_window_filter(finished_indices_inverse)[:100]

        # Combine recommendations into a dictionary for separate processing
        wasserstein_recommendations = {
            "Control": finished_indices_control,
            "Exp": finished_indices_exp,
            "Linear": finished_indices_linear,
            "Inverse": finished_indices_inverse
        }
        # Create the folder based on the label
        # Step 6: Prepare the result DataFrame for this function set
        label = df.loc[relevant_index, 'Label']  # Get the label for the situation
        match_name = df.loc[relevant_index, 'match_name']
        time_of_situation = str(datetime.timedelta(seconds=df.loc[relevant_index, 'Time [s]']))

        folder_name = f"{label}-{match_name}-{time_of_situation}"
        generate_folder(folder_name)

        # Save the time and match_name to a text file
        with open(f"{folder_name}/situation-info.txt", "w") as f:
                f.write(f"Match Name: {match_name}\n")
                f.write(f"Time of Situation: {time_of_situation}\n")

        # Step 5: Process each set of indices separately
        for func_name, indices_set in wasserstein_recommendations.items():
            if len(indices_set) == 0:
                continue  # Skip if no indices in this set

            # Step 5.1: Compute similar movements for the current set
            similar_movement = find_similar_movement(df, relevant_index, indices_set,length_of_sequence=120)
            similar_movement_ranked = sorted(similar_movement, key=lambda x: x[0])

            # Step 5.2: Select indices, ensuring no two are within 100 places of each other
            selected_indices = []
            for _, index in similar_movement_ranked:
                if all(abs(index - selected) > 500 for selected in selected_indices):
                    selected_indices.append(index)
                if len(selected_indices) == 10:  # Stop once we have 10 indices
                    break

            new_df = df.loc[selected_indices[:10]]

            
            


            # Generate the result DataFrame
            result_df = new_df[["match_name", "Time [s]", "half_team"]]
            result_df["Time Min"] = result_df["Time [s]"].apply(lambda x: str(datetime.timedelta(seconds=x)).replace("_","-"))

            # Save the ground truth CSV for this function set
            result_df.to_csv(f"{folder_name}/{folder_name}-{func_name}-ground-truth.csv")

            # Shuffle and save the ordering truth CSV for this function set
            result_df.sample(frac=1).to_csv(f"{folder_name}/{folder_name}-{func_name}-for-ordering-truth.csv")

            # Store the recommendations in the dictionary
            recommendations_dict[relevant_index] = recommendations_dict.get(relevant_index, {})
            recommendations_dict[relevant_index][func_name] = selected_indices[:10]

    # Return the dictionary with recommendations
    return recommendations_dict

    

reccomendations_dict = process_situations(df)

[143547, 154347, 181797, 189572, 199447, 208722, 214138]
2
2
23722
21284
2
2
23722
21284
2
2
23722
21284
2
2
23722
21284
Folder 'run-into-space-Denmark_England-0:00:10' does not exist.


/tmp/ipykernel_51434/3725379782.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df["Time Min"] = result_df["Time [s]"].apply(lambda x: str(datetime.timedelta(seconds=x)).replace("_","-"))
/tmp/ipykernel_51434/3725379782.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df["Time Min"] = result_df["Time [s]"].apply(lambda x: str(datetime.timedelta(seconds=x)).replace("_","-"))
/tmp/ipykernel_51434/3725379782.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

2
2
23722
21284
2
2
23722
21284
2
2
23722
21284
2
2
23722
21284
Folder 'breakthrough-wb-Denmark_England-0:07:22' does not exist.


/tmp/ipykernel_51434/3725379782.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df["Time Min"] = result_df["Time [s]"].apply(lambda x: str(datetime.timedelta(seconds=x)).replace("_","-"))
/tmp/ipykernel_51434/3725379782.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df["Time Min"] = result_df["Time [s]"].apply(lambda x: str(datetime.timedelta(seconds=x)).replace("_","-"))
/tmp/ipykernel_51434/3725379782.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

2
2
23722
21284
2
2
23722
21284
2
2
23722
21284
2
2
23722
21284
Folder 'breakthrough-space-Denmark_England-0:25:40' does not exist.


...

/tmp/ipykernel_51434/3725379782.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df["Time Min"] = result_df["Time [s]"].apply(lambda x: str(datetime.timedelta(seconds=x)).replace("_","-"))
/tmp/ipykernel_51434/3725379782.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df["Time Min"] = result_df["Time [s]"].apply(lambda x: str(datetime.timedelta(seconds=x)).replace("_","-"))
/tmp/ipykernel_51434/3725379782.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

2
2
23722
21284
2
2
23722
21284
2
2
23722
21284
2
2
23722
21284
Folder 'run-into-space-Denmark_England-0:43:37' does not exist.


/tmp/ipykernel_51434/3725379782.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df["Time Min"] = result_df["Time [s]"].apply(lambda x: str(datetime.timedelta(seconds=x)).replace("_","-"))
/tmp/ipykernel_51434/3725379782.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df["Time Min"] = result_df["Time [s]"].apply(lambda x: str(datetime.timedelta(seconds=x)).replace("_","-"))
/tmp/ipykernel_51434/3725379782.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

2
2
23722
21284
2
2
23722
21284
2
2
23722
21284
2
2
23722
21284
Folder 'breakthrough-even-Denmark_England-0:46:06' does not exist.


/tmp/ipykernel_51434/3725379782.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df["Time Min"] = result_df["Time [s]"].apply(lambda x: str(datetime.timedelta(seconds=x)).replace("_","-"))
/tmp/ipykernel_51434/3725379782.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df["Time Min"] = result_df["Time [s]"].apply(lambda x: str(datetime.timedelta(seconds=x)).replace("_","-"))
/tmp/ipykernel_51434/3725379782.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

In [9]:
reccomendations_dict

{143547: {'Control': [282444,
   146652,
   234456,
   106164,
   252492,
   91176,
   241632,
   169836,
   31800,
   31236],
  'Exp': [282432,
   39012,
   172536,
   137148,
   261552,
   204288,
   278052,
   121056,
   22116,
   214176],
  'Linear': [282432,
   137148,
   261552,
   204288,
   22116,
   230304,
   283008,
   146652,
   11364,
   79956],
  'Inverse': [39012,
   172548,
   137148,
   261552,
   204288,
   121056,
   22116,
   214176,
   88464,
   230304]},
 154347: {'Control': [139692,
   173868,
   137736,
   246408,
   187848,
   52500,
   8664,
   143796,
   186996,
   63072],
  'Exp': [139692,
   173868,
   137736,
   246396,
   187848,
   52500,
   203160,
   97896,
   8676,
   12348],
  'Linear': [139692,
   173868,
   137736,
   246396,
   187848,
   52500,
   8664,
   143796,
   12348,
   186996],
  'Inverse': [139692,
   173868,
   137736,
   246396,
   187848,
   52500,
   203160,
   97896,
   272820,
   8688]},
 181797: {'Control': [199524,
   176352,
   

In [ ]:
root ="csvs_for_ranking"
folders = os.listdir(root)



print(folders)
import re

def extract_seconds_from_folder(folder_name):
    # Use regex to find the time pattern (hh:mm:ss)
    match = re.search(r'(\d+):(\d+):(\d+)', folder_name)
    if match:
        hours, minutes, seconds = map(int, match.groups())
        # Convert to total seconds
        return hours * 3600 + minutes * 60 + seconds
    else:
        # Return None if no time is found
        return None



for folder in folders:
    csvs = os.listdir(root + "/"+folder)
    for csv in csvs:
        if ("ground-truth" in csv):
            print(folder)


            seconds = extract_seconds_from_folder(folder)

            if (seconds > 2700 and "England" in csv):
                myplayer = VideoPlayer("3-2024-2036178-Denmark_England.mp4",folder,979+2773+(seconds-2700),0,[])
            if (seconds < 2700 and "England" in csv):
                myplayer = VideoPlayer("3-2024-2036178-Denmark_England.mp4",folder,8+seconds,0,[])
            if (seconds > 2700 and "Slovenia" in csv):
                myplayer = VideoPlayer("3-2024-2036165-slovenia-denmark.mp4",folder,2769+954+(seconds-2700),0,[])
            if (seconds < 2700 and "Slovenia" in csv):
                myplayer = VideoPlayer("3-2024-2036165-slovenia-denmark.mp4",folder,8+seconds,0,[])
            
            
            
            df = pd.read_csv(root +"/"+folder+"/"+csv)
            #Match_name, time, half
            indices = df[["Time [s]","half_team",'Unnamed: 0', 'match_name']]
            ratings = []
            matches = df.match_name.unique()
            for match_name in matches:
                if "England" in match_name: 
                    myplayer = VideoPlayer("3-2024-2036178-Denmark_England.mp4",folder,8,979+2773,indices[indices['match_name'] == match_name].to_numpy())
                    [ratings.append(row) for row in myplayer.ratings]
                if "Slovenia" in match_name:
                    myplayer = VideoPlayer("3-2024-2036165-slovenia-denmark.mp4",folder,8,2769+954,indices[indices['match_name'] == match_name].to_numpy())
                    [ratings.append(row) for row in myplayer.ratings]
            if (len(ratings) > 0):
                ratings_df  = pd.DataFrame(ratings, columns=['Unnamed: 0',"rating"])
                rated_situations = pd.merge(df, ratings_df, how ="inner", on=['Unnamed: 0'])
                rated_situations =rated_situations.drop_duplicates('Unnamed: 0')
                rated_situations.to_csv(root +"/"+folder+"/"+"ranked-"+csv)




['breakthrough-even-Denmark_England-0:46:06']
breakthrough-even-Denmark_England-0:46:06


[00007f3894004ea0] gl gl: Initialized libplacebo v4.192.1 (API v192)
[00007f387c001720] gl gl: Initialized libplacebo v4.192.1 (API v192)
invalid command name "139882041258304update_time_label"
    while executing
"139882041258304update_time_label"
    ("after" script)
[00007f386c001720] gl gl: Initialized libplacebo v4.192.1 (API v192)
invalid command name "139882041360576update_time_label"
    while executing
"139882041360576update_time_label"
    ("after" script)


breakthrough-even-Denmark_England-0:46:06


[00007f38c00019c0] gl gl: Initialized libplacebo v4.192.1 (API v192)
invalid command name "139882041365120update_time_label"
    while executing
"139882041365120update_time_label"
    ("after" script)
[00007f3860001720] gl gl: Initialized libplacebo v4.192.1 (API v192)
invalid command name "139882041361920update_time_label"
    while executing
"139882041361920update_time_label"
    ("after" script)
[00007f384c004010] gl gl: Initialized libplacebo v4.192.1 (API v192)
invalid command name "139882041367424update_time_label"
    while executing
"139882041367424update_time_label"
    ("after" script)


breakthrough-even-Denmark_England-0:46:06


[00007f38700015d0] gl gl: Initialized libplacebo v4.192.1 (API v192)
invalid command name "139882041370368update_time_label"
    while executing
"139882041370368update_time_label"
    ("after" script)
Exception in Tkinter callback
Traceback (most recent call last):
  File "/home/martin/anaconda3/envs/DBU/lib/python3.10/tkinter/__init__.py", line 1921, in __call__
    return self.func(*args)
  File "/home/martin/Desktop/Thesis Preperation/Thesis_V1/Thesis_V1/evaluation/../libs/Video_Player.py", line 101, in next_time
    if self.distance_index_list.any():
AttributeError: 'list' object has no attribute 'any'
[00007f3824001700] gl gl: Initialized libplacebo v4.192.1 (API v192)
invalid command name "139882041370752update_time_label"
    while executing
"139882041370752update_time_label"
    ("after" script)
[00007f38a40332b0] gl gl: Initialized libplacebo v4.192.1 (API v192)
invalid command name "139882041490688update_time_label"
    while executing
"139882041490688update_time_label"
    (

breakthrough-even-Denmark_England-0:46:06


[00007f381c001720] gl gl: Initialized libplacebo v4.192.1 (API v192)
invalid command name "139882041373952update_time_label"
    while executing
"139882041373952update_time_label"
    ("after" script)
[00007f387c10b780] gl gl: Initialized libplacebo v4.192.1 (API v192)
invalid command name "139882041372928update_time_label"
    while executing
"139882041372928update_time_label"
    ("after" script)


In [11]:
ratings_df  = pd.DataFrame(ratings, columns=['Unnamed: 0',"rating"])

NameError: name 'ratings' is not defined

In [ ]:
rated_situations = pd.merge(df, ratings_df, how ="inner", on=['Unnamed: 0'])

In [ ]:
rated_situations =rated_situations.drop_duplicates('Unnamed: 0')

In [ ]:
print(rated_situations)

    Unnamed: 0       match_name  Time [s] half_team        Time Min  rating
0       179537  Denmark_England   1449.60        1H  0:24:09.600000       1
1       174689  Denmark_England   1255.68        1H  0:20:55.680000       0
3       199505  Denmark_England   2248.32        1H  0:37:28.320000       1
5       149921  Denmark_England    264.96        1H  0:04:24.960000       1
6       193697  Denmark_England   2016.00        1H         0:33:36       0
7       208001  Denmark_England   2588.16        1H  0:43:08.160000       1
8       238961  Denmark_England   3758.92        2H  1:02:38.920000       0
9       253697  Denmark_England   4348.36        2H  1:12:28.360000       0
10      176321  Denmark_England   1320.96        1H  0:22:00.960000       1
11      185441  Denmark_England   1685.76        1H  0:28:05.760000       1
